In [151]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [248]:
from experimentation import data_preparation
from experimentation import regression_OLS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
df1 = data_preparation(file_location = "uk_data.csv", splits = 2, lags = 1)
df2 = regression_OLS(file_location = "uk_data.csv", splits = 2, lags = 1, train_share = 0.8, p_cutoff = 1.00)

/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [242]:
list_ = list(df1.keys())

In [250]:
df2

{'split_1_oos_r2': 0.10534681895228104,
 'split_1_oos_mae': 0.5057817386693055,
 'split_1_oos_mse': 0.3745035362778092,
 'split_1_oos_rmse': 0.6119669405105224,
 'split_1_oos_mape': inf,
 'split_1_summary': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                    gdp   R-squared:                       0.125
 Model:                            OLS   Adj. R-squared:                  0.071
 Method:                 Least Squares   F-statistic:                     2.321
 Date:                Sun, 26 Nov 2023   Prob (F-statistic):             0.0505
 Time:                        20:34:08   Log-Likelihood:                -74.399
 No. Observations:                  87   AIC:                             160.8
 Df Residuals:                      81   BIC:                             175.6
 Df Model:                           5                                         
 Covariance Type:       